In [1]:
# load packages
import os
import sys
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
tf.__version__

'1.10.0'

# Load Data

In [4]:
data = pd.read_csv("../DATAHUB/iris-dataset.csv")

In [5]:
data.shape

(150, 6)

In [6]:
data.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


In [7]:
target_values = data.Species.unique()
target_values

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [8]:
# create mapping for target labels
target_values_mapping = dict()
i = 0
for tar in target_values:
    if tar not in target_values_mapping:
        target_values_mapping[tar] = i
        i += 1
    else:
        continue

In [9]:
target_values_mapping

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [10]:
# convert target labels into categorical values
data.Species = data.Species.apply(lambda x: target_values_mapping[x])
data.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0


## Converting To Binary Classification

In [11]:
data = data[data.Species <= 1]
data.shape

(100, 6)

In [12]:
data.Species.unique()

array([0, 1])

In [13]:
features = np.array(data.iloc[:, 1:-1]) # load features into a numpy array
features.shape

(100, 4)

In [14]:
labels = np.array(data.iloc[:, [-1]]) # load target values into a numpy array
labels.shape

(100, 1)

In [15]:
features[0], labels[0]

(array([5.1, 3.5, 1.4, 0.2]), array([0]))

In [16]:
features[1], labels[1]

(array([4.9, 3. , 1.4, 0.2]), array([0]))

In [17]:
# split data into train and validate set
val_split_size = 0.2
random_indices = np.random.rand(len(features)) <= val_split_size

train_x = features[~random_indices]
train_y = labels[~random_indices]

val_x = features[random_indices]
val_y = labels[random_indices]

In [18]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

((78, 4), (78, 1), (22, 4), (22, 1))

# Vectorized Logistic Regression Model

In [19]:
# placeholders
x = tf.placeholder(dtype=tf.float32, shape=(None, features.shape[1]), name="feature_x")
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target_y")

In [20]:
# define weights and bias
w = tf.Variable(tf.random_normal(shape=(1, features.shape[1]), dtype=tf.float32, mean=0.0, stddev=0.1))
b = tf.Variable(tf.zeros(shape=(1, 1), dtype=tf.float32))

In [21]:
logistic_model = tf.sigmoid(tf.add(tf.matmul(x, tf.transpose(w)), b))

In [22]:
# loss to minimize
loss_function = tf.reduce_mean(-(y*tf.log(logistic_model) + (1 - y)*tf.log(1 - logistic_model))) # binary cross entropy

## Using Inbuilt Optimizer

In [23]:
trainer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss_function) # define optimizer over loss function

In [24]:
# train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("[Training...]")

    for epoch in range(10):
        sess.run(trainer, feed_dict={x:train_x, y:train_y})
        train_loss = sess.run(loss_function, feed_dict={x:train_x, y:train_y})
        print("Step:", epoch, "\tTrain Loss:", train_loss, end="\t")
        val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
        print("Val Loss:", val_loss)
    
    print("Final Weights:", sess.run(w))
    predicted_y = sess.run(logistic_model, feed_dict={x:val_x, y:val_y})

[Training...]
Step: 0 	Train Loss: 0.66107166	Val Loss: 0.70484805
Step: 1 	Train Loss: 0.6873195	Val Loss: 0.8089012
Step: 2 	Train Loss: 0.6566027	Val Loss: 0.781486
Step: 3 	Train Loss: 0.56897557	Val Loss: 0.66378504
Step: 4 	Train Loss: 0.4748255	Val Loss: 0.52935845
Step: 5 	Train Loss: 0.40851447	Val Loss: 0.42647317
Step: 6 	Train Loss: 0.37489048	Val Loss: 0.36724922
Step: 7 	Train Loss: 0.3533003	Val Loss: 0.33261317
Step: 8 	Train Loss: 0.32356444	Val Loss: 0.3002415
Step: 9 	Train Loss: 0.28337237	Val Loss: 0.26403475
Final Weights: [[-0.1951803 -0.3899726  0.5876911  0.616648 ]]


## Prediction Analysis

In [25]:
# clip the prediction on threshold of 0.5 to get final prediction
final_prediction = list()
for pred in predicted_y:
    if pred > 0.5:
        final_prediction.append(1)
    else:
        final_prediction.append(0)
final_prediction = np.array(final_prediction).reshape((len(final_prediction), 1))
final_prediction.shape, val_y.shape

((22, 1), (22, 1))

In [26]:
# find accuracy
correct_prediciton = 0
for i in range(val_y.shape[0]):
    if final_prediction[i] == val_y[i]:
        correct_prediciton += 1
    else:
        continue
"Accuracy = ", correct_prediciton / val_y.shape[0] * 100

('Accuracy = ', 100.0)

## Use Your Own Gradient Descent

In [27]:
# compute gradients
grad_w, grad_b = tf.gradients(xs=[w, b], ys=loss_function)

# update
new_w = w.assign(w - 0.1*grad_w)
new_b = b.assign(b - 0.1*grad_b)

In [28]:
# train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("[Training...]")

    for epoch in range(10):
        _, _, train_loss = sess.run([new_w, new_b, loss_function], feed_dict={x:train_x, y:train_y})
        print("Step:", epoch, "\tTrain Loss:", train_loss, end="\t")
        val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
        print("Val Loss:", val_loss)
    
    print("Final Weights:", sess.run(w))
    predicted_y = sess.run(logistic_model, feed_dict={x:val_x, y:val_y})

[Training...]
Step: 0 	Train Loss: 0.6862202	Val Loss: 0.6793507
Step: 1 	Train Loss: 0.6364179	Val Loss: 0.62880987
Step: 2 	Train Loss: 0.597692	Val Loss: 0.59168476
Step: 3 	Train Loss: 0.56273824	Val Loss: 0.556361
Step: 4 	Train Loss: 0.5308453	Val Loss: 0.52447164
Step: 5 	Train Loss: 0.50170666	Val Loss: 0.49534023
Step: 6 	Train Loss: 0.47505248	Val Loss: 0.46873626
Step: 7 	Train Loss: 0.4506366	Val Loss: 0.44439095
Step: 8 	Train Loss: 0.4282361	Val Loss: 0.4220727
Step: 9 	Train Loss: 0.40764976	Val Loss: 0.40157378
Final Weights: [[-0.0784956  -0.2247189   0.4368278   0.21662694]]


## Prediction Analysis

In [29]:
# clip the prediction on threshold of 0.5 to get final prediction
final_prediction = list()
for pred in predicted_y:
    if pred > 0.5:
        final_prediction.append(1)
    else:
        final_prediction.append(0)
final_prediction = np.array(final_prediction).reshape((len(final_prediction), 1))
final_prediction.shape, val_y.shape

((22, 1), (22, 1))

In [30]:
# find accuracy
correct_prediciton = 0
for i in range(val_y.shape[0]):
    if final_prediction[i] == val_y[i]:
        correct_prediciton += 1
    else:
        continue
"Accuracy = ", correct_prediciton / val_y.shape[0] * 100

('Accuracy = ', 100.0)

Perfect Score!